# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2025-08-19 13:12:05.400037: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Import Data

In [2]:
df = pd.read_csv('sudoku.csv')
df

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...
...,...,...
999995,3000280000290000300054001077402030980086070031...,3175289464291768356854391277462135989586472131...
999996,0030006000040860057000009409350407208067200502...,5234976811942863757685139429356417288167294532...
999997,0003508200618040300500090000700600029030070100...,7493568212618745393582197468749613529235876146...
999998,0702006900030400010000650205600300000947005800...,4752816936239478511893657245628341793947165828...


# Prepare Data

In [3]:
def str81_to_grid(s):
    # Trả về mảng (9,9) int8
    arr = np.array(list(s), dtype=np.int8)
    return arr.reshape(9, 9)

def one_hot_10_for_input(grid):
    # 10 kênh: kênh 0 = ô trống, kênh 1..9 = chữ số tương ứng
    # grid chứa 0..9
    x = np.zeros((9, 9, 10), dtype=np.float32)
    # kênh "trống"
    x[:, :, 0] = (grid == 0).astype(np.float32)
    # kênh 1..9
    for d in range(1, 10):
        x[:, :, d] = (grid == d).astype(np.float32)
    return x

def one_hot_9_for_target(solution_grid):
    # 9 kênh (1..9) — không có '0' trong nghiệm
    y = np.zeros((9, 9, 9), dtype=np.float32)
    for d in range(1, 10):
        y[:, :, d-1] = (solution_grid == d).astype(np.float32)
    return y

# Biến đổi toàn bộ dataset -> (N, 9, 9, 10) và (N, 9, 9, 9)
X_list, y_list, W_list = [], [], []
for q, s in zip(df['quizzes'].values, df['solutions'].values):
    qg = str81_to_grid(q)
    sg = str81_to_grid(s)
    X_list.append(one_hot_10_for_input(qg))
    y_list.append(one_hot_9_for_target(sg))
    # Trọng số: ưu tiên ô trống (thường ~2–5 lần). Ở đây chọn 5.0
    w = np.ones((9, 9), dtype=np.float32)
    w[qg == 0] = 5.0
    W_list.append(w)

X = np.stack(X_list)  # (N,9,9,10)
y = np.stack(y_list)  # (N,9,9,9)
W = np.stack(W_list)  # (N,9,9)

# Split the data

In [4]:
TRAIN_RATIO, VAL_RATIO = 0.8, 0.1
BATCH_SIZE, SEED = 256, 123

n = len(X)
idx = tf.random.shuffle(tf.range(n), seed=42)
n_train = int(n * TRAIN_RATIO)
n_val   = int(n * VAL_RATIO)
n_test  = n - n_train - n_val

idx_train = idx[:n_train].numpy()
idx_val   = idx[n_train:n_train+n_val].numpy()
idx_test  = idx[n_train+n_val:].numpy()

X_train, y_train, W_train = X[idx_train], y[idx_train], W[idx_train]
X_val,   y_val, W_val   = X[idx_val],   y[idx_val], W[idx_val]
X_test,  y_test, W_test  = X[idx_test],  y[idx_test], W[idx_test]

train = (tf.data.Dataset.from_tensor_slices((X_train, y_train, W_train))
            .shuffle(buffer_size=n_train, seed=SEED, reshuffle_each_iteration=True)
            .batch(BATCH_SIZE)
            .prefetch(tf.data.AUTOTUNE))

val  = (tf.data.Dataset.from_tensor_slices((X_val, y_val, W_val))
           .batch(BATCH_SIZE)
           .prefetch(tf.data.AUTOTUNE))

test = (tf.data.Dataset.from_tensor_slices((X_test, y_test, W_test))
           .batch(BATCH_SIZE)
           .prefetch(tf.data.AUTOTUNE))

I0000 00:00:1755609248.873587    1206 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22260 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:03:00.0, compute capability: 8.9


# Build model

In [5]:
def build_cnn():
    inp = keras.Input(shape=(9, 9, 10))

    # Khối conv cơ bản
    x = layers.Conv2D(128, 3, padding='same', use_bias = False)(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 3, padding='same', use_bias = False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Một vài khối dư nhẹ
    for _ in range(8):
        res = x
        x = layers.Conv2D(128, 3, padding='same', use_bias = False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(128, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(layers.add([x, res]))

    # 1x1 conv để ra 9 lớp (mỗi ô 9 xác suất)
    x = layers.Conv2D(9, 1, padding='same')(x)
    out = layers.Softmax(axis=-1)(x)  # softmax theo 9 lớp

    model = keras.Model(inp, out)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    return model

model = build_cnn()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 9, 9, 10)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 9, 9, 128) │     11,520 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 9, 9, 128) │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 9, 9, 128) │    147,456 │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 9, 9, 128) │    147,456 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 9, 9, 128) │    147,584 │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
│                     │                   │            │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 9, 9, 128) │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 9, 9, 128) │    147,456 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 9, 9, 128) │    147,584 │ re_lu_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
│                     │                   │            │ activation[0][0]

 Total params: 2,529,673 (9.65 MB)

 Trainable params: 2,525,065 (9.63 MB)

 Non-trainable params: 4,608 (18.00 KB)

# Train model

In [6]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True, monitor='val_loss'),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-5)
]

history = model.fit(
    train,
    validation_data=val,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50


2025-08-19 13:14:56.010322: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fef6c034eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-19 13:14:56.010357: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-08-19 13:14:56.245539: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-19 13:14:57.620835: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200


   9/3125 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - accuracy: 0.1369 - loss: 17.1180

I0000 00:00:1755609301.399866    1693 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 79s 19ms/step - accuracy: 0.8896 - loss: 1.3938 - val_accuracy: 0.9599 - val_loss: 0.5483 - learning_rate: 0.0010
Epoch 2/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 61s 18ms/step - accuracy: 0.9766 - loss: 0.3206 - val_accuracy: 0.9743 - val_loss: 0.3624 - learning_rate: 0.0010
Epoch 3/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9844 - loss: 0.2152 - val_accuracy: 0.9829 - val_loss: 0.2394 - learning_rate: 0.0010
Epoch 4/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9884 - loss: 0.1607 - val_accuracy: 0.9858 - val_loss: 0.2022 - learning_rate: 0.0010
Epoch 5/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9909 - loss: 0.1270 - val_accuracy: 0.9882 - val_loss: 0.1694 - learning_rate: 0.0010
Epoch 6/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9926 - loss: 0.1035 - val_accuracy: 0.9901 - val_loss: 0.1433 - learning_rate: 0.0010
Epoch 7/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9938 

# Evaluate

In [7]:
test_loss, test_acc = model.evaluate(test, verbose=0)
print(f"Test (weighted) loss: {test_loss:.4f} | acc: {test_acc:.4f}")

Test (weighted) loss: 0.0771 | acc: 0.9962


# Test

In [8]:
def solve_with_argmax(puzzle_str):
    qg = str81_to_grid(puzzle_str)
    x = one_hot_10_for_input(qg)[None, ...]  # (1,9,9,10)
    pred = model.predict(x, verbose=0)[0]    # (9,9,9) softmax
    grid_pred = np.argmax(pred, axis=-1) + 1 # 1..9
    # Trả lại chuỗi 81 ký tự của nghiệm dự đoán
    return ''.join(str(d) for d in grid_pred.flatten())
print(solve_with_argmax(df['quizzes'].iloc[0]))

864371259325849761971265843436192587198657432257483916689734125713528694542916378
